## Configuracion

In [0]:
from config_loader import Config

paths = Config.get_paths()
params = Config.get_params()

catalog = paths['catalog']
schemas = paths['schemas']
tables = paths['tables']

target_col = params['feature_engineering']['target_column']
features_list = params['feature_engineering']['features']
pk_col = params['feature_engineering']['primary_key']
seed = params['feature_engineering']['seed']
split_ratio = [
    params['feature_engineering']['train_ratio'], 
    params['feature_engineering']['test_ratio']
]

## Extraer Data

In [0]:
source_path = f"{catalog}.{schemas['curated']}.{tables['cdz']['df_curated']}"
df = spark.read.table(source_path)
display(df)

## Train y Test

In [0]:
train_df, test_df = df.randomSplit(split_ratio, seed=seed)
train_df.count()
test_df.count()

## Feature Selection

In [0]:
# para correr la definicion de la funcion, directamente la llamo aca

%run "./feature_engineering"

In [0]:
selected_columns = [pk_col] + features_list + [target_col]

train_fs = train_df.select(*selected_columns)
test_fs = test_df.select(*selected_columns)

# crear una notebook q se pasa un df y te lo trae transformado (notebook)

## Guardar en Feature Store

In [0]:
train_target = f"{catalog}.{schemas['feature_store']}.{tables['features']['train_set']}"
test_target = f"{catalog}.{schemas['feature_store']}.{tables['features']['test_set']}"

train_fs.write.format("delta").mode("overwrite").saveAsTable(train_target)
test_fs.write.format("delta").mode("overwrite").saveAsTable(test_target)